# **Custom Minigrid**

In [70]:
import sys
import os

#10_DQL_Applied/Minigrid_with_Monsters

# Agrega el directorio principal a sys.path
module_path = os.path.abspath(os.path.join('./Minigrid_with_Monsters'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [59]:
from __future__ import annotations


from Minigrid_with_Monsters.minigrid.core.constants import COLOR_NAMES
from Minigrid_with_Monsters.minigrid.core.grid import Grid
from Minigrid_with_Monsters.minigrid.core.mission import MissionSpace
from Minigrid_with_Monsters.minigrid.core.world_object import Goal
from Minigrid_with_Monsters.minigrid.minigrid_env import MiniGridEnv
from Minigrid_with_Monsters.minigrid.core.world_object import Monster
from time import sleep

import numpy as np
from typing import Any


class LockedRoomEnv(MiniGridEnv):
    def __init__(self, size=19, max_steps: int | None = None, **kwargs):
        self.size = size
        self.monster = Monster()
        self.goal_pos = None

        if max_steps is None:
            max_steps = 10 * size

        mission_space = MissionSpace(mission_func=lambda: "Go to the green goal square avoid the red monster.")
        
        super().__init__(
            mission_space=mission_space,
            width=size,
            height=size,
            max_steps=max_steps,
            **kwargs,
        )

    def _gen_grid(self, width, height):
        # Create the grid
        self.grid = Grid(width, height)

        # Generate the surrounding walls
        self.grid.horz_wall(0, 0)
        self.grid.horz_wall(0, height - 1)
        self.grid.vert_wall(0, 0)
        self.grid.vert_wall(width - 1, 0)

        room_w = width // 2
        room_h = height // 2

        # For each row of rooms
        for j in range(0, 2):
            # For each column
            for i in range(0, 2):
                xL = i * room_w
                yT = j * room_h
                xR = xL + room_w
                yB = yT + room_h

                # Bottom wall and door
                if i + 1 < 2:
                    self.grid.vert_wall(xR, yT, room_h)
                    pos = (xR, self._rand_int(yT + 1, yB))
                    self.grid.set(*pos, None)

                # Bottom wall and door
                if j + 1 < 2:
                    self.grid.horz_wall(xL, yB, room_w)
                    pos = (self._rand_int(xL + 1, xR), yB)
                    self.grid.set(*pos, None)

        # Place the agent goal and monster
        self.place_agent()
        self.goal_pos = self.place_obj(Goal())
        self.monster.position = self.place_obj(self.monster)

    def step(self, action):
        obs, reward, terminated, truncated, info = super().step(action)

        # Check if monster can see the agent. If so, move towards it.
        # If the monster is in the same cell as the agent, the agent is caught.
        if self.monster.can_see(self.grid, self.agent_pos):
            # if the monster can see the agent, move towards it
            self.monster.move_towards(self.grid, self.agent_pos)

            if self.monster.position == self.agent_pos:
                reward = -.5
                terminated = True
        else:
            # if the monster can't see the agent, move randomly
            self.monster.patrol_forward(self.grid)

        self.grid.set(*self.monster.position, self.monster)

        if self.agent_pos == self.goal_pos:
            reward = 1
            terminated = True


        return obs, reward, terminated, truncated, info



# **Play**

In [50]:
import gymnasium as gym
from gymnasium.utils.play import play
import numpy as np

env = LockedRoomEnv(render_mode="rgb_array", max_steps=1000)

play(env, keys_to_action={
    (ord("a"),): 0,
    (ord("d"),): 1,
    (ord("w"),): 2,
    (ord("j"),): 3,
    (ord("k"),): 5,
}, noop=6)

# **El Modelo PPO: Cómo Ayuda al Agente a Jugar el Juego MiniGrid**

El modelo que usaremos para jugar este juego de MiniGrid se llama **Optimización de Política Proximal (PPO)**. Es un algoritmo popular de aprendizaje automático que enseña a un agente cómo navegar por una cuadrícula, alcanzar un objetivo y evitar obstáculos aprendiendo de sus experiencias.

## **1. El Extractor de Características**

### **¿Qué es un Extractor de Características?**
Antes de que el agente pueda tomar decisiones, necesita entender lo que está viendo en la cuadrícula. Un **extractor de características** ayuda con esto al enfocarse en los detalles importantes del entorno.

### **¿Cómo Funciona?**
- El extractor de características utiliza **Redes Neuronales Convolucionales (CNNs)**, que están especializadas en analizar datos visuales.
- **Las CNNs** funcionan escaneando una imagen para detectar patrones, como bordes o formas, que son cruciales para entender la disposición de la cuadrícula.

### **Por Qué Es Importante:**
Este paso simplifica los datos visuales, facilitando que el agente aprenda y tome decisiones inteligentes en el juego.

---


In [65]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch

class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(MinigridFeaturesExtractor, self).__init__(observation_space, features_dim)
        
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
        )
        
        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]
        
        self.linear = nn.Sequential(
            nn.Linear(n_flatten, features_dim),
            nn.ReLU()
        )
    
    def forward(self, observations):
        return self.linear(self.cnn(observations))

In [66]:
policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)


## **2. Inicialización de PPO**

### **¿Qué es la Inicialización de PPO?**
Ahora que el agente puede entender su entorno, configuramos el modelo **Optimización de Política Proximal (PPO)**, que guiará las acciones del agente.

### **¿Qué Sucede Aquí?**
- **Política:** Elegimos una **CnnPolicy**, lo que significa que las decisiones del agente se basan en los datos visuales procesados por la CNN.
- **Extractor de Características:** Las características procesadas por la CNN se introducen en el modelo PPO para ayudar al agente a decidir su próximo movimiento.
- **Hiperparámetros:** Se ajustan configuraciones importantes como la tasa de aprendizaje para controlar qué tan rápido aprende el agente.

### **Por Qué Es Importante:**
Esta configuración le da al agente la capacidad de aprender de sus experiencias y mejorar su desempeño en el juego con el tiempo.

---

## **3. Entrenamiento del Modelo PPO**

### **¿Qué es el Entrenamiento?**
Con el modelo configurado, es hora de entrenar al agente. **El entrenamiento** es donde el agente aprende interactuando con la cuadrícula, probando diferentes acciones y descubriendo qué funciona mejor.

### **¿Cómo Funciona?**
- El agente realiza movimientos y recibe recompensas o penalizaciones según sus acciones.
- A lo largo de muchos intentos, el agente aprende las mejores estrategias para maximizar sus recompensas.
- El entrenamiento continúa hasta que el agente se convierte en un experto en alcanzar el objetivo.

### **Por Qué Es Importante:**
El entrenamiento transforma al agente de un principiante a un profesional en jugar el juego MiniGrid, ayudándole a tomar mejores decisiones y alcanzar sus objetivos de manera constante.


In [67]:
from stable_baselines3 import PPO
from Minigrid_with_Monsters.minigrid.wrappers import ImgObsWrapper

# Initialize the environment
env = LockedRoomEnv(max_steps=500)
env = ImgObsWrapper(env)

# Initialize the model with custom feature extractor
model = PPO(
    "CnnPolicy", 
    env, 
    policy_kwargs=policy_kwargs, 
    learning_rate=2e-4,  # Lower the learning rate
    gamma=0.99,          # Adjust the discount factor
    clip_range=0.1,      # Lower the clip range
    verbose=1
)


# Train the model
model.learn(8e4)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 315      |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 973      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 274         |
|    ep_rew_mean          | -0.214      |
| time/                   |             |
|    fps                  | 707         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.012304688 |
|    clip_fraction        | 0.0346      |
|    clip_range      


# **Generar un Video y Mejorar el Modelo**

En la siguiente sección, vamos a generar un video que muestra cómo el agente se desempeña en el entorno MiniGrid. 

Mira el video para observar cómo actúa el agente. Después, intenta entrenar el modelo nuevamente para ver si puedes mejorar su rendimiento. ¡Recuerda que el aprendizaje es un proceso continuo!

In [69]:
import cv2
import numpy as np

# Create the environment and wrap it again for rendering
env = LockedRoomEnv(render_mode="rgb_array", max_steps=500)
env = ImgObsWrapper(env)

# Load the trained model
#model = PPO.load("locked_room_model.zip")

# Reset the environment and get the initial observation
obs, _ = env.reset()  # Extract only the observation

done = False

# Set up the video writer (using OpenCV)
frame_width = env.render().shape[1]
frame_height = env.render().shape[0]
video = cv2.VideoWriter('ppo_agent_performance.avi', cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

time_step = 0
while time_step < 1000:
    # Get action from model
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)

    # Render the environment and collect the frame
    frame = env.render()
    
    # Convert RGB to BGR (OpenCV uses BGR format)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Write the frame into the video
    video.write(frame)

    time_step += 1

    if done:
        obs, _ = env.reset()

# Release the video writer
video.release()


# Close the environment
env.close()



# **Guardar el Modelo**

Si el modelo ha mostrado un buen rendimiento, ¡asegúrate de guardarlo! Esto te permitirá usarlo más adelante sin tener que volver a entrenarlo.

Puedes guardar el modelo con el siguiente comando en tu notebook:

```python
model.save("nombre_del_modelo")
```

¡Así podrás seguir mejorando tu modelo o usarlo para otros experimentos en el futuro!

In [61]:
# Save the model
model.save("locked_room_model")